# NLP Basic Assignment
## NLP 맛보기 - spam.csv를 가지고 유의미한 해석을 도출해주세요!

In [1]:
# 필요한 모듈 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다. 물론 7주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고, 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다 :)

In [2]:
spam = pd.read_csv('spam.csv')

In [3]:
spam.iloc[5]['v2']

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [4]:
spam.v1 = spam.v1.apply(lambda x : 0 if x == 'ham' else 1) # spam을 1 ham 을 0으로 변환해준다.

In [5]:
spam.v1.value_counts()[1]/ len(spam)

0.13406317300789664

In [17]:
test = spam.copy()

## Tokenizing

In [25]:
spam.v2[5569]

'pity    was in mood for that  so   any other suggestions '

In [26]:
import re
def cleanText(x) :
    x_ = re.sub('[^a-zA-Z]', ' ', x).lower() # 특수문자 제거 & 소문자
    return x_

spam.v2 = spam.v2.apply(cleanText)

In [27]:
spam.v2[5569]

'pity    was in mood for that  so   any other suggestions '

In [28]:
spam.v2.nunique()

5147

In [29]:
test = spam.copy()
test1 = spam.copy()

### 불용어 처리 및 Tokenizing

In [32]:
import nltk

# 예시 코드 코드
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords') # nltk에 저장되어 있는 불용어를 불러온다.
english_stops = list(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [33]:
english_stops = list(stopwords.words('english'))
token = []

for sentence in test.v2:
    words = word_tokenize(sentence)
    words = [word for word in words if word not in english_stops]
    token.append(words)

test.v2 = token

In [34]:
test.v2

0       [go, jurong, point, crazy, available, bugis, n...
1                          [ok, lar, joking, wif, u, oni]
2       [free, entry, wkly, comp, win, fa, cup, final,...
3           [u, dun, say, early, hor, u, c, already, say]
4          [nah, think, goes, usf, lives, around, though]
                              ...                        
5567    [nd, time, tried, contact, u, u, pound, prize,...
5568                      [b, going, esplanade, fr, home]
5569                            [pity, mood, suggestions]
5570    [guy, bitching, acted, like, interested, buyin...
5571                                   [rofl, true, name]
Name: v2, Length: 5572, dtype: object

In [38]:
from tokenizers import SentencePieceBPETokenizer

with open('test.txt', 'w', encoding= 'utf8') as f :
    for row in test1.itertuples() :
        f.write(row.v2 + '\n')

In [39]:
sp = SentencePieceBPETokenizer()
sp.train(files = 'test.txt', min_frequency = 1, vocab_size = 2000)

In [40]:
sp = SentencePieceBPETokenizer()
sp.train(files = 'test.txt', min_frequency = 1, vocab_size = 2000)

In [41]:
sp.get_vocab_size()

2000

In [42]:
word2idx = sp.get_vocab()
idx2word = {j:i for i,j in word2idx.items()}

In [43]:
def getlist(x) :
    tokens = sp.encode(x).tokens
    results = [token for token in tokens if token != '▁'] #의미없는 띄어쓰기는 제거 해준다.
    return results

In [44]:
test1.v2 = test1.v2.apply(getlist)

In [45]:
test1.v2

0       [▁go, ▁until, ▁j, ur, ong, ▁point, ▁crazy, ▁av...
1                [▁ok, ▁lar, ▁jo, king, ▁wif, ▁u, ▁on, i]
2       [▁free, ▁entry, ▁in, ▁a, ▁wkly, ▁comp, ▁to, ▁w...
3       [▁u, ▁dun, ▁say, ▁so, ▁early, ▁hor, ▁u, ▁c, ▁a...
4       [▁nah, ▁i, ▁don, ▁t, ▁think, ▁he, ▁goes, ▁to, ...
                              ...                        
5567    [▁this, ▁is, ▁the, ▁nd, ▁time, ▁we, ▁have, ▁tr...
5568    [▁will, ▁b, ▁going, ▁to, ▁es, pl, an, ade, ▁fr...
5569    [▁p, ity, ▁was, ▁in, ▁mo, od, ▁for, ▁that, ▁so...
5570    [▁the, ▁guy, ▁did, ▁some, ▁bit, ch, ing, ▁but,...
5571           [▁ro, f, l, ▁its, ▁true, ▁to, ▁its, ▁name]
Name: v2, Length: 5572, dtype: object

## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram, GloVe, FastText가 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

cv1 = CountVectorizer(max_features= 500, stop_words = 'english', lowercase= False)
cv2 = CountVectorizer(max_features= 500, stop_words = 'english', lowercase= False)

In [47]:
tokens_ham = sum(test[test.v1 == 0]['v2'], []) # CountVectorizer는 리스트 형태로 값이 들어와야 하기 때문에 이중 리스트를 flatten 시켜준다.
tokens_spam = sum(test[test.v1 == 1]['v2'], [])

In [48]:
tdm_ham = cv1.fit_transform(tokens_ham)
tdm_spam = cv2.fit_transform(tokens_spam)

## 본인이 도출해낸 해석을 적어주세요!

- 유사도, Wordcloud, 이진 분류 모델, Plot 뭐든 상관없으니 분명하고 인상적인 해석을 적어주시면 됩니다.